## triplet

In [1]:
import json

with open("rag_truth_train2.json", "r") as f:
    train_data = json.load(f)
with open("rag_truth_dev.json", "r") as f:
    dev_data = json.load(f)
with open("rag_truth_test.json", "r") as f:
    test_data = json.load(f)

In [2]:
# prefixをつける
def add_prefix(data):
    for d in data:
        d["text"] = "Please judge the following statement as true or false based on the references above: " + d["text"]
    return data

train_data = add_prefix(train_data)
dev_data = add_prefix(dev_data)
test_data = add_prefix(test_data)

In [3]:
# task_type: QA, Data2txt, Summary
task_name = "Summary"
train_data = [d for d in train_data if d["task_type"] == task_name]
dev_data = [d for d in dev_data if d["task_type"] == task_name]
test_data = [d for d in test_data if d["task_type"] == task_name]

In [3]:
import random

def create_trip(data, id_list):
    trip = []
    for id in id_list:
        num = 0
        no_hal = []
        has_hal = []
        for d in data:
            if num == 6:
                num = 0
                if no_hal == [] or has_hal == []:
                    break
                '''
                for no in no_hal:
                    for has in has_hal:
                        trip.append({"anchor":ref,"positive": no, "negative": has, "labels":[0,1]})
                '''
                # シャッフル
                random.seed(id)
                no_hal = random.sample(no_hal, len(no_hal))
                has_hal = random.sample(has_hal, len(has_hal))
                
                if len(no_hal)==1 or len(no_hal)==5:
                    trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels":[0,1]})
                elif len(no_hal)==2 or len(no_hal)==4:
                    trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels":[0,1]})
                    trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels":[0,1]})
                elif len(no_hal)==3:
                    trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels":[0,1]})
                    trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels":[0,1]})
                    trip.append({"anchor":ref,"positive": no_hal[2], "negative": has_hal[2], "labels":[0,1]})
                no_hal = []
                has_hal = []
                break
            elif d["source_id"] == id:
                num +=1
                ref = d["ref"]
                if d["labels"] == 0:
                    no_hal.append(d["text"])
                else:
                    has_hal.append(d["text"])
        if num == 6:
            if len(no_hal)==1 or len(no_hal)==5:
                trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels":[0,1]})
            elif len(no_hal)==2 or len(no_hal)==4:
                trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels":[0,1]})
                trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels":[0,1]})
            elif len(no_hal)==3:
                trip.append({"anchor":ref,"positive": no_hal[0], "negative": has_hal[0], "labels":[0,1]})
                trip.append({"anchor":ref,"positive": no_hal[1], "negative": has_hal[1], "labels":[0,1]})
                trip.append({"anchor":ref,"positive": no_hal[2], "negative": has_hal[2], "labels":[0,1]})
    return trip

In [4]:
train_id = [d["source_id"] for d in train_data]
train_id = list(set(train_id))
dev_id = [d["source_id"] for d in dev_data]
dev_id = list(set(dev_id))
test_id = [d["source_id"] for d in test_data]
test_id = list(set(test_id))
print(len(train_id), len(dev_id), len(test_id))
train_trip = create_trip(train_data, train_id)
dev_trip = create_trip(dev_data, dev_id)
test_trip = create_trip(test_data, test_id)

2305 210 450


In [5]:
len(train_trip), len(dev_trip), len(test_trip)

(3760, 337, 633)

In [6]:
from datasets import Dataset, DatasetDict
import pandas as pd

train_df = pd.DataFrame(train_trip)
dev_df = pd.DataFrame(dev_trip)
test_df = pd.DataFrame(test_trip)

train_ds = Dataset.from_pandas(train_df)
dev_ds = Dataset.from_pandas(dev_df)
test_ds = Dataset.from_pandas(test_df)

tri_raw_datasets = DatasetDict({"train": train_ds, "dev": dev_ds, "test": test_ds})
tri_raw_datasets

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 3760
    })
    dev: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 337
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 633
    })
})

In [7]:
#!huggingface-cli login

In [8]:
from transformers import AutoTokenizer, AutoModel

#tri_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/RoBERTa-base")
tri_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
#tri_tokenizer.pad_token_id = tri_tokenizer.eos_token_id
#tri_tokenizer.padding_side = "left"
#tri_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large",use_fast=False)


In [9]:
tri_raw_datasets["train"][0]

{'anchor': '{\'name\': \'Santa Barbara Chicken Ranch\', \'address\': \'2618 De La Vina St\', \'city\': \'Santa Barbara\', \'state\': \'CA\', \'categories\': \'Health & Medical, Restaurants, Mexican, Cannabis Clinics, Barbeque\', \'hours\': {\'Monday\': \'11:0-22:0\', \'Tuesday\': \'11:0-22:0\', \'Wednesday\': \'11:0-22:0\', \'Thursday\': \'11:0-22:0\', \'Friday\': \'11:0-22:0\', \'Saturday\': \'11:0-22:0\', \'Sunday\': \'11:0-22:0\'}, \'attributes\': {\'BusinessParking\': {\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}, \'RestaurantsReservations\': False, \'OutdoorSeating\': True, \'WiFi\': \'no\', \'RestaurantsTakeOut\': True, \'RestaurantsGoodForGroups\': True, \'Music\': None, \'Ambience\': {\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}}, \'business_stars\': 4.0, \'review_info\': [{\'review_stars\': 5.0, \'re

In [10]:
from transformers import DataCollatorWithPadding

def tri_tokenize_function(examples):
    anchor = tri_tokenizer(examples["anchor"], truncation=True,max_length=512)
    positive = tri_tokenizer(examples["positive"], truncation=True,max_length=512)
    negative = tri_tokenizer(examples["negative"], truncation=True,max_length=512)

    return {
        "anchor_input_ids": anchor["input_ids"],
        "anchor_attention_mask": anchor["attention_mask"],
        "positive_input_ids": positive["input_ids"],
        "positive_attention_mask": positive["attention_mask"],
        "negative_input_ids": negative["input_ids"],
        "negative_attention_mask": negative["attention_mask"],
    }

tri_tokenized_datasets = tri_raw_datasets.map(tri_tokenize_function, batched=True)
tri_tokenized_datasets = tri_tokenized_datasets.remove_columns(["anchor", "positive", "negative"])
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tri_tokenized_datasets.set_format("torch")
tri_data_collator = DataCollatorWithPadding(tokenizer=tri_tokenizer)

2025-01-22 20:37:01.314695: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 20:37:01.772531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-22 20:37:02.011261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-22 20:37:02.011701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 20:37:02.400546: I tensorflow/core/platform/cpu_feature_gua

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

Map:   0%|          | 0/633 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorWithPadding
from torch.nn.utils.rnn import pad_sequence
import torch

class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        # features の例: [{'anchor_input_ids': ..., 'anchor_attention_mask': ..., ...}, ...]
        
        # データのリストを作成
        anchor_ids = [x['anchor_input_ids'].clone().detach() for x in features]
        positive_ids = [x['positive_input_ids'].clone().detach() for x in features]
        negative_ids = [x['negative_input_ids'].clone().detach() for x in features]
        
        anchor_mask = [x['anchor_attention_mask'].clone().detach() for x in features]
        positive_mask = [x['positive_attention_mask'].clone().detach() for x in features]
        negative_mask = [x['negative_attention_mask'].clone().detach() for x in features]
        
        # パディング処理
        anchor_ids = pad_sequence(anchor_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        positive_ids = pad_sequence(positive_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        negative_ids = pad_sequence(negative_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        
        anchor_mask = pad_sequence(anchor_mask, batch_first=True, padding_value=0)
        positive_mask = pad_sequence(positive_mask, batch_first=True, padding_value=0)
        negative_mask = pad_sequence(negative_mask, batch_first=True, padding_value=0)

        # ラベルを作成
        labels = [x['labels'] for x in features]
        
        
        # バッチ辞書を作成
        batch = {
            "input_ids": [anchor_ids, positive_ids, negative_ids],
            "attention_mask": [anchor_mask, positive_mask, negative_mask],
            "labels": labels
        }
        
        return batch


tri_data_collator = CustomDataCollator(tokenizer=tri_tokenizer)

In [12]:
from transformers import AutoModel


#base_model = AutoModel.from_pretrained("FacebookAI/RoBERTa-base")
base_model = AutoModel.from_pretrained("microsoft/Phi-3.5-mini-instruct")
#base_model = AutoModel.from_pretrained("vinai/bertweet-large")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
import torch

# 使う装置
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
base_model.to(device)
device

device(type='cuda')

In [14]:
import torch
import torch.nn as nn
from transformers.modeling_outputs import ModelOutput
import os


class TripletModel(nn.Module):
    def __init__(self, base_model, loss_function, question_encoder=None, generator=None):
        super(TripletModel, self).__init__()
        self.base_model = base_model
        #self.triplet_loss_fn = nn.TripletMarginLoss(margin=1.0, p=2)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(base_model.config.hidden_size * 2, 2)  # ラベル予測用の分類層
        # self.classification_loss_fn = nn.CrossEntropyLoss()  # ラベル予測の損失関数
        self.loss_function = loss_function
        self.question_encoder = question_encoder
        self.generator = generator
        #self.alpha = alpha

    def forward(
        self,
        input_ids,
        attention_mask = None,
        labels = None,
    ):
        
        anchor_input_ids = input_ids[0]
        positive_input_ids = input_ids[1]
        negative_input_ids = input_ids[2]
        anchor_attention_mask = attention_mask[0]
        positive_attention_mask = attention_mask[1]
        negative_attention_mask = attention_mask[2]


        anchor_output = self.base_model(input_ids=anchor_input_ids, attention_mask=anchor_attention_mask,return_dict=True)[0][:,-1,:]
        positive_output = self.base_model(input_ids=positive_input_ids, attention_mask=positive_attention_mask,return_dict=True)[0][:,-1,:]
        negative_output = self.base_model(input_ids=negative_input_ids, attention_mask=negative_attention_mask,return_dict=True)[0][:,-1,:]

        #anchor_output = anchor_output.mean(dim=1)
        #positive_output = positive_output.mean(dim=1)
        #negative_output = negative_output.mean(dim=1)

        anchor_output = self.dropout(anchor_output)
        positive_output = self.dropout(positive_output)
        negative_output = self.dropout(negative_output)

        # ラベル予測の出力
        positive_logits = self.classifier(torch.cat([anchor_output, positive_output], dim=1))
        negative_logits = self.classifier(torch.cat([anchor_output, negative_output], dim=1))

        classification_loss, triplet_loss=self.loss_function(anchor_output, positive_output, negative_output, positive_logits, negative_logits)
        loss = classification_loss + triplet_loss
        
        #return ModelOutput(logits=[positive_logits, negative_logits], loss=loss,classification_loss=classification_loss,triplet_loss=triplet_loss)
        return ModelOutput(logits=[positive_logits, negative_logits],loss=loss)

    def save_pretrained(self, save_directory):
        """
        モデル全体とサブモジュールを保存するカスタムメソッド
        """
        os.makedirs(save_directory, exist_ok=True)

        # サブモジュールの保存
        if self.question_encoder is not None:
            self.question_encoder.save_pretrained(os.path.join(save_directory, "question_encoder"))
        if self.generator is not None:
            self.generator.save_pretrained(os.path.join(save_directory, "generator"))

        # モデル全体の重みを保存
        torch.save(self.state_dict(), os.path.join(save_directory, "pytorch_model.bin"))

    @classmethod
    def from_pretrained(cls, save_directory):
        """
        保存済みモデルをロードするカスタムメソッド
        """
        question_encoder = None
        generator = None

        # サブモジュールのロード
        if os.path.exists(os.path.join(save_directory, "question_encoder")):
            question_encoder = AutoModel.from_pretrained(os.path.join(save_directory, "question_encoder"))
        if os.path.exists(os.path.join(save_directory, "generator")):
            generator = AutoModel.from_pretrained(os.path.join(save_directory, "generator"))

        # モデル全体の重みをロード
        model = cls(question_encoder=question_encoder, generator=generator)
        model.load_state_dict(torch.load(os.path.join(save_directory, "pytorch_model.bin")))

        return model

In [15]:
import torch.nn.functional as F
import torch.nn as nn

clas_list = []
trip_list = []
def triplet_loss(anchor_output, positive_output, negative_output, positive_logits, negative_logits):
    # ラベル予測の損失計算
    positive_targets = torch.zeros(positive_output.size(0), dtype=torch.long).to(device)  # ラベル1 (positive)
    negative_targets = torch.ones(negative_output.size(0), dtype=torch.long).to(device)
    positive_loss = nn.CrossEntropyLoss()(positive_logits, positive_targets)
    negative_loss = nn.CrossEntropyLoss()(negative_logits, negative_targets)

    # ラベル予測の損失を平均
    classification_loss = (positive_loss + negative_loss) / 2.0

    # トリプレット損失の計算
    #triplet_loss = nn.TripletMarginLoss(margin=1, p=2)(anchor_output, positive_output, negative_output)
    triplet_loss_fn = (nn.TripletMarginWithDistanceLoss(margin=1.5,distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y)))
    triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)
    # 最終損失
    total_loss = classification_loss + triplet_loss 
    #return total_loss
    trip_list.append(triplet_loss)
    clas_list.append(classification_loss)
    return classification_loss, triplet_loss

In [16]:
import torch
import numpy as np
# negativeがhalluでlabelが1
def compute_tri_metrics(eval_pred):
    #print("aa")
    # モデルの出力から損失と logits を取得
    logits,labels= eval_pred
    
    # ロジットのサイズに合わせて推論を行う
    positive_logits = torch.tensor(logits[0])
    negative_logits = torch.tensor(logits[1])
    #positive_logits = torch.tensor(logits[0][0])
    #negative_logits = torch.tensor(logits[0][1])



    # ラベル予測
    positive_preds = torch.argmax(positive_logits, dim=1)
    negative_preds = torch.argmax(negative_logits, dim=1)

    # 正解と予測ラベルの一致数
    correct_positive = (positive_preds == 0).sum().item()
    correct_negative = (negative_preds == 1).sum().item()
    
    # ラベルの数
    total_samples = positive_preds.size(0) + negative_preds.size(0)
    
    # PositiveとNegativeの予測数
    positive_preds_num = (positive_preds == 1).sum().item()
    negative_preds_num = (negative_preds == 1).sum().item()
    positive_num = positive_preds.size(0)
    negative_num = negative_preds.size(0)

    #print(negative_preds_num,positive_preds_num,negative_num)

    # Precision, Recall, F1の計算
    precision = negative_preds_num / (positive_preds_num + negative_preds_num) if (positive_preds_num + negative_preds_num) > 0 else 0
    recall = negative_preds_num / negative_num if negative_num > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # 平均損失と正解率の計算
    #avg_loss = loss.item() if loss is not None else 0
    accuracy = (correct_positive + correct_negative) / total_samples
    return {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1": f1
    }


In [17]:
from transformers import TrainingArguments
from transformers import Trainer

from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding
import torch
from models import TripletModel

"""
base_model = AutoModel.from_pretrained("microsoft/Phi-3.5-mini-instruct")
# モデルとトークナイザーを読み込む
name = "./1224_triplet"
tri_model = TripletModel.from_pretrained(base_model, triplet_loss, name)
#tokenizer = AutoTokenizer.from_pretrained(name)

# 使う装置
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tri_model.to(device)
"""

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="steps",  
    save_steps=10000,
    learning_rate=1e-6,# 1e-6
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16 = True,
    gradient_accumulation_steps=12,
    logging_dir="./logs",
    remove_unused_columns=False,
    report_to="tensorboard",
    optim="adafactor",
)

tri_model = TripletModel(base_model, triplet_loss)
#triplet_loss_logger = TripletLossLogger()

trainer = Trainer(
    model=tri_model,
    args=training_args,
    train_dataset=tri_tokenized_datasets["train"],
    eval_dataset=tri_tokenized_datasets["dev"],
    data_collator=tri_data_collator,
    tokenizer=tri_tokenizer,
    compute_metrics=compute_tri_metrics,

)

/gs/fs/tga-arase-student/yamada/nlp/lib64/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/22090.1.interactive/ipykernel_4194085/3761690756.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:

trainer.evaluate()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_bmm)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Recall,Precision,F1
0,1.988000,1.565921,0.004600,0.692878,0.676558,0.699387,0.687783
1,1.488800,1.380101,0.004600,0.725519,0.816024,0.690955,0.748299


In [ ]:
trainer.evaluate(eval_dataset=tri_tokenized_datasets["test"])

In [20]:
def create_dev_task(name):
    dev_data2 = [d for d in test_data if d["task_type"] == name]
    dev_id2 = [d["source_id"] for d in dev_data2]
    dev_id2 = list(set(dev_id2))
    dev_trip2 = create_trip(dev_data2, dev_id2)
    dev_df2 = pd.DataFrame(dev_trip2)
    dev_ds2 = Dataset.from_pandas(dev_df2)
    tri_tokenized_datasets_task = dev_ds2.map(tri_tokenize_function, batched=True)
    tri_tokenized_datasets_task = tri_tokenized_datasets_task.remove_columns(["anchor", "positive", "negative"])
    tri_tokenized_datasets_task.set_format("torch")
    return tri_tokenized_datasets_task


In [21]:
dev_qa = create_dev_task("QA")

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [22]:
trainer.evaluate(eval_dataset=dev_qa)

{'eval_loss': 0.5213183760643005,
 'eval_model_preparation_time': 0.0039,
 'eval_accuracy': 0.8966666666666666,
 'eval_recall': 0.92,
 'eval_precision': 0.8789808917197452,
 'eval_f1': 0.8990228013029316,
 'eval_runtime': 9.4958,
 'eval_samples_per_second': 15.796,
 'eval_steps_per_second': 4.002}

In [23]:
dev_d2t = create_dev_task("Data2txt")

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

In [24]:
trainer.evaluate(eval_dataset=dev_d2t)

{'eval_loss': 0.9073538780212402,
 'eval_model_preparation_time': 0.0039,
 'eval_accuracy': 0.7852233676975945,
 'eval_recall': 0.7491408934707904,
 'eval_precision': 0.8074074074074075,
 'eval_f1': 0.7771836007130126,
 'eval_runtime': 19.65,
 'eval_samples_per_second': 14.809,
 'eval_steps_per_second': 3.715}

In [25]:
dev_sum = create_dev_task("Summary")

Map:   0%|          | 0/192 [00:00<?, ? examples/s]

In [26]:
trainer.evaluate(eval_dataset=dev_sum)

{'eval_loss': 1.1023415327072144,
 'eval_model_preparation_time': 0.0039,
 'eval_accuracy': 0.71875,
 'eval_recall': 0.65625,
 'eval_precision': 0.75,
 'eval_f1': 0.7,
 'eval_runtime': 12.0214,
 'eval_samples_per_second': 15.971,
 'eval_steps_per_second': 3.993}

In [21]:
# トレーニング後にモデルを保存
# トレーニング後にモデルを保存
name = "./1228_triplet"
trainer.save_model(name)
trainer.save_state()
tri_model.save_pretrained(name)

In [19]:
import torch
torch.cuda.empty_cache() 
!nvidia-smi

Tue Nov 26 00:10:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100                    On  | 00000000:84:00.0 Off |                    0 |
| N/A   29C    P0             151W / 699W |  15370MiB / 95830MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
# alpha探索
import optuna
from transformers import AutoModel
import transformers
import torch
import peft

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# 目的関数を定義
def objective(trial):
    torch.cuda.empty_cache() 
    # alpha を 0.1 刻みで探索
    alpha = trial.suggest_categorical('alpha', [0.3])
    
    # モデルの損失関数に alpha を設定
    base_model = AutoModel.from_pretrained("microsoft/Phi-3.5-mini-instruct")
    base_model.to(device)

    tri_model = TripletModel(base_model,triplet_loss, alpha=alpha)

    # モデルの学習
    trainer = Trainer(
        model=tri_model,
        args=training_args,
        train_dataset=tri_tokenized_datasets["train"],
        eval_dataset=tri_tokenized_datasets["dev"],
        data_collator=tri_data_collator,
        tokenizer=tri_tokenizer,
        compute_metrics=compute_tri_metrics,
    
    )
    trainer.train()

    # 評価 (ここでは eval_accuracy を最大化する例)
    metrics = trainer.evaluate()
    # 返すスコアを変更可能 (例: F1 スコア, 精度など)
    return metrics['eval_f1']

# Optuna のスタディを作成
study = optuna.create_study(direction='maximize')  # 精度を最大化する設定
study.optimize(objective, n_trials=2)  # 試行回数 (0.1 刻みなら11試行)

# 最適な alpha の確認
best_alpha = study.best_params['alpha']
print(f"Best alpha: {best_alpha}")


[I 2024-12-21 22:07:52,130] A new study created in memory with name: no-name-3c0841e6-4883-4e79-a10b-1410de3e8fc1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/2105545.1.all.q/ipykernel_661295/3443566117.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
0,0.937700,0.666758,0.732036,0.688623,0.754098,0.719875
1,0.651100,0.606497,0.772455,0.745509,0.787975,0.766154
2,0.586500,0.558576,0.778443,0.730539,0.807947,0.767296
3,0.521100,0.537769,0.799401,0.784431,0.808642,0.796353
4,0.476900,0.541523,0.799401,0.835329,0.779330,0.806358
5,0.427400,0.532387,0.797904,0.754491,0.826230,0.788732
6,0.394300,0.531311,0.800898,0.763473,0.825243,0.793157
7,0.356700,0.534783,0.794910,0.805389,0.788856,0.797037
8,0.330600,0.537783,0.799401,0.802395,0.797619,0.800000
9,0.304900,0.543543,0.787425,0.778443,0.792683,0.785498


[I 2024-12-21 23:25:06,386] Trial 0 finished with value: 0.785498489425982 and parameters: {'alpha': 0.3}. Best is trial 0 with value: 0.785498489425982.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/2105545.1.all.q/ipykernel_661295/3443566117.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
0,0.969700,0.738462,0.694611,0.595808,0.742537,0.661130
1,0.682100,0.656266,0.758982,0.676647,0.810036,0.737357
2,0.613800,0.571792,0.763473,0.751497,0.769939,0.760606
3,0.554700,0.551639,0.779940,0.724551,0.814815,0.767036
4,0.511200,0.558430,0.790419,0.730539,0.829932,0.777070
5,0.456200,0.559521,0.785928,0.712575,0.835088,0.768982
6,0.421900,0.555264,0.782934,0.775449,0.787234,0.781297
7,0.375700,0.543227,0.790419,0.772455,0.801242,0.786585
8,0.347300,0.542052,0.797904,0.760479,0.822006,0.790047
9,0.317500,0.543016,0.797904,0.772455,0.813880,0.792627


[I 2024-12-22 00:42:59,919] Trial 1 finished with value: 0.7926267281105991 and parameters: {'alpha': 0.3}. Best is trial 1 with value: 0.7926267281105991.


Best alpha: 0.3


In [20]:
import transformers
base_model = AutoModel.from_pretrained("microsoft/Phi-3.5-mini-instruct",
        torch_dtype=torch.float16,
        quantization_config=transformers.BitsAndBytesConfig(  # 量子化する指定
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
    )

import re
# どこにLoRAのパラメータを追加できるか
linear_layers = list(set(re.findall(
    r'\((\w+)\): Linear', str(base_model.modules)
)))
print(linear_layers)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['qkv_proj', 'gate_up_proj', 'o_proj', 'down_proj']


In [ ]:
# すべての試行結果を取得
import matplotlib.pyplot as plt

alphas = [trial.params['alpha'] for trial in study.trials]
scores = [trial.value for trial in study.trials]

plt.plot(alphas, scores, marker='o')
plt.xlabel('Alpha')
plt.ylabel('Evaluation Score (e.g., Accuracy)')
plt.title('Alpha vs Evaluation Score')
plt.grid()
plt.show()


### 色々検証

In [29]:
import pandas as pd

# 予測結果をDataFrameに変換
predictions_df = pd.DataFrame(predictions.predictions[1])

predictions_df['label_ids'] = 1

# CSVファイルとして保存
predictions_df.to_csv("predictions_1.csv", index=False)

In [30]:
import pandas as pd

# CSVファイルを読み込む
df1 = pd.read_csv("predictions.csv")
df2 = pd.read_csv("predictions_1.csv")

# 縦に結合
combined_df = pd.concat([df1, df2], ignore_index=True)

# 結合したデータをCSVとして保存
combined_df.to_csv("predictions_2.csv", index=False)


In [ ]:
trip_list = [x.item() for x in trip_list]
clas_list = [x.item() for x in clas_list]

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(0,len(clas_list),len(clas_list)//1000)
clas_list = np.array(clas_list)
trip_list = np.array(trip_list)
plt.plot(x,clas_list[x])

In [ ]:
plt.plot(x,trip_list[x])

In [ ]:
plt.plot(x,trip_list[x]+clas_list[x])

In [27]:
print(len(predictions.predictions[0]))  # 出力の形状を確認
print(len(predictions.label_ids[0]))

2509
1255


In [39]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# CSVファイルを読み込む
df = pd.read_csv("predictions_2.csv")

# クラスラベルを予測
df['predicted_label'] = np.where(df["0"] > df["1"], 0, 1)

# 正解ラベルと予測ラベルを取得
true_labels = df['label_ids']  # 正解ラベルの列名を指定
predicted_labels = df['predicted_label']

# 各評価指標を計算
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# 結果を表示
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.6853
Precision: 0.6286
Recall: 0.9059
F1 Score: 0.7422
